In [7]:
import pandas as pd

df_all_results = pd.read_csv("all_backtest_results.csv")


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

# Load combined backtest results
df_all_results = pd.read_csv("all_backtest_results.csv")

# Split by strategy
df_vader_results = df_all_results[df_all_results['strategy'] == 'VADER'].copy()
df_lr_results = df_all_results[df_all_results['strategy'] == 'Logistic Regression'].copy()
df_finbert_results = df_all_results[df_all_results['strategy'] == 'FinBERT'].copy()


# ===============================
# Comparative analysis
# ===============================
def analyze_strategy_differences():
    """
    Analyze why FinBERT outperformed or underperformed.
    """

    # 1. Trade quality proxy (price movement quality)
    print("\nTRADE QUALITY ANALYSIS")
    print("=" * 60)

    for name, df in zip(
        ['VADER', 'Logistic Regression', 'FinBERT'],
        [df_vader_results, df_lr_results, df_finbert_results]
    ):
        daily_returns = df['portfolio_value'].pct_change().dropna()
        print(
            f"{name}: "
            f"Avg Daily Return = {daily_returns.mean()*100:.3f}%, "
            f"Volatility = {daily_returns.std()*100:.3f}%"
        )

    # 2. False signal analysis (losing days)
    print("\nFALSE SIGNAL ANALYSIS")
    print("=" * 60)

    vader_losses = (df_vader_results['portfolio_value'].pct_change() < 0).sum()
    lr_losses = (df_lr_results['portfolio_value'].pct_change() < 0).sum()
    finbert_losses = (df_finbert_results['portfolio_value'].pct_change() < 0).sum()

    print(f"VADER losing days: {vader_losses}")
    print(f"LR losing days: {lr_losses}")
    print(f"FinBERT losing days: {finbert_losses}")

    # 3. Strategy consistency (rolling Sharpe)
    print("\nSTRATEGY CONSISTENCY ANALYSIS")
    print("=" * 60)

    rolling_window = 20

    def rolling_sharpe(df):
        returns = df['portfolio_value'].pct_change()
        return (
            returns.rolling(rolling_window).mean() /
            returns.rolling(rolling_window).std()
        ) * np.sqrt(252)

    plt.figure(figsize=(10, 5))
    plt.plot(rolling_sharpe(df_vader_results), label='VADER')
    plt.plot(rolling_sharpe(df_lr_results), label='Logistic Regression')
    plt.plot(rolling_sharpe(df_finbert_results), label='FinBERT')
    plt.title("Rolling Sharpe Ratio Comparison")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()


# ===============================
# Statistical significance testing
# ===============================
def test_significance(returns1, returns2):
    """
    Test if performance difference is statistically significant.
    """
    t_stat, p_value = stats.ttest_ind(
        returns1,
        returns2,
        equal_var=False
    )
    return t_stat, p_value


# Calculate daily returns
returns_vader = df_vader_results['portfolio_value'].pct_change().dropna()
returns_lr = df_lr_results['portfolio_value'].pct_change().dropna()
returns_finbert = df_finbert_results['portfolio_value'].pct_change().dropna()

# Test FinBERT vs VADER
t_stat, p_value = test_significance(returns_finbert, returns_vader)

print("\nStatistical Significance Test (FinBERT vs VADER)")
print("=" * 60)
print(f"T-statistic: {t_stat:.3f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Result: FinBERT significantly outperforms VADER (p < 0.05)")
else:
    print("Result: No significant difference (p > 0.05)")


# Run analysis
analyze_strategy_differences()


NameError: name 'returns_finbert' is not defined